In [1]:
# 最后修改 2019-04-09

In [1]:
from mongoengine import *
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime

使用** mongoengine **  
[connect](http://docs.mongoengine.org/guide/connecting.html#guide-connecting)进行数据库连接

In [2]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## S1 获取指定目录下的全部文件名称

In [23]:
targetfilename=r"CH2000BST.txt"
# mac 15
# targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data"
# targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/data/typhoon"
# mac 16
targetpath=r"/Users/drno/Documents/01proj/TyphoonSearchSys/data/typhoon"

# aw15
# targetpath=r"D:\01proj\typhoon\TyphoonSearchSys\demo_data"
# p52s
# targetpath=r"D:\04git仓库\TyphoonSearchSys\demo_data"
# w540
# targetpath=r"D:\02git仓库\TyphoonSearchSys\demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [24]:
list_files=os.listdir(targetpath)
list_files[:5]

['CH1998BST.txt',
 'CH1986BST.txt',
 'CH1994BST.txt',
 'CH1951BST.txt',
 'CH2012BST.txt']

获取以.txt结尾的

In [25]:
import re

In [26]:
list_files[0]

'CH1998BST.txt'

In [27]:
print(re.match('^.+.txt$','.DS_Store'))

None


过滤.txt结尾的文件列表

In [28]:
list_filted=list(filter(lambda x:re.match('^.+.txt$',x),list_files))

In [29]:
list_filted[:5]

['CH1998BST.txt',
 'CH1986BST.txt',
 'CH1994BST.txt',
 'CH1951BST.txt',
 'CH2012BST.txt']

## 1 读取文件

### 读取方式1：使用pandas

In [30]:
fullname

'/Users/drno/Documents/01proj/TyphoonSearchSys/data/typhoon/CH2000BST.txt'

不使用pandas的read_table

In [31]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\s+',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


--------

## 2 判断标志位的方式（必须同时满足），并获取标志位所在位置的数组（mark_indexs）

1-第一位是否为66666

2- 第7位是否不为Nan

In [32]:
data.iloc[0][0]==66666 and data.iloc[0][7]!=np.nan

True

### 写成方法：

In [33]:
def checkMark(index):
    '''
        判断是否为标志位
    '''
    return data.iloc[index][0]==66666 and data.iloc[index][7]!=np.nan

找到标志位所在的位置

In [34]:
mark_indexs=[]
for i in range(len(data)):
#     print(i)
    if checkMark(i):
        mark_indexs.append(i)

--------

## 3 获取观测值并写入mongoDB（暂时先写在list中）

In [35]:
# 保存起止数组的数组
list_startend=[]
index=0
for val in mark_indexs:
#     list_startend.append(mark_indexs[index:2])
#     print(mark_indexs[index:index+2])
    list_startend.append(mark_indexs[index:index+2])
    index=index+1
#     print(index)

[[789, 830], [830]]  
注意最后的数组是长度为1的数组，因为最后一次出现标志位后出现的都是观测数据，便结束了

----

## 4 使用mongoEngine

In [36]:
from mongoengine import *

In [37]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

定义 mongodb中的存储对象（ORM）

In [38]:
class GeoTyphoonRealData(Document):
    '''
        支持geojson的存储至mongodb的model
    '''
    code=StringField(max_length=10)
    num=StringField()
    date=DateTimeField()
    bp=FloatField()
    wsm=FloatField()
    # 注意此处与django中的类型不同，django的类型为IntegerField，mongoengine为IntField！
    level=IntField()
    # latlon=models.ForeignKey(Point,on_delete=models.CASCADE)
    latlon=PointField()
    meta = {'collection': 'geotyphoonrealdata'}
    
    def __str__(self):
        return f'code:{self.code}|num:{self.num}|date:{self.date}|bp:{self.bp}|wsm:{self.wsm}|level:{self.level}|latlon:{self.latlon}'

In [39]:
import datetime

** 提示错误如下 **:  
NotUniqueError: Tried to save duplicate unique keys (E11000 duplicate key error collection: typhoon.Typhoon_geotyphoonrealdata index: __primary_key__ dup key: { : null })  
是mongodb中的对应document中可能设置了__primary_key__，我的解决办法是 ** 删除掉 ** 

### S1 根据标志位遍历df

循环遍历list_startend

In [40]:
for val in list_startend:
    if len(val)>1:        
        print(f"起始{val[0]}——终止{val[1]-1}")
    elif len(val)==1:
        print(f"起始{val[0]}")

起始0——终止31
起始32——终止47
起始48——终止58
起始59——终止68
起始69——终止78
起始79——终止106
起始107——终止144
起始145——终止181
起始182——终止203
起始204——终止238
起始239——终止245
起始246——终止288
起始289——终止329
起始330——终止363
起始364——终止371
起始372——终止397
起始398——终止416
起始417——终止473
起始474——终止499
起始500——终止520
起始521——终止537
起始538——终止566
起始567——终止589
起始590——终止614
起始615——终止643
起始644——终止674
起始675——终止715
起始716


In [41]:
def getHeaderBody(start,end=None):
    '''
        根据df的起止位置截取其中的头部（66666	0	25	0	1	0	6.0	(nameless)	20180501.0），
        以及数据内容body（2017041406	0	109	1302	1010	10）
    '''
    header=data.iloc[start-1]
    body=data.iloc[start:end,0:6]
    return header,body

In [42]:
from datetime import datetime

- 19-04-03 此处加入一个新的需求，加入对于 **台风 数字编号 **的获取

In [43]:
# data.iloc[0]

In [55]:
def convert2Typhoon(obj,code,num):
    '''
        根据传入的series将其转为typhoob Model
    '''
    lat=float(f"{str(obj[2])[:-1]}.{str(obj[2])[-1:]}")
    lon=float(f"{str(obj[3])[:-1]}.{str(obj[3])[-1:]}")
    stamp_str=obj[0]
    # TODO 注意此处需要修改num的值
    stamp=datetime.strptime(str(stamp_str),'%Y%m%d%H')
#     num_str=str(stamp_str)[2:4]+str('%02d'%num)
    num_str=str(num).zfill(4)
    typhoon_temp=GeoTyphoonRealData(code=code,
                                date=stamp,
                                num=num_str,
                                bp=obj[4],
                                wsm=obj[5],
                                level=obj[1],
                                latlon=[lon,lat])
    return typhoon_temp

### S2 拼接经纬度
获取经纬度：  
    - 将字符串转成float，并且最后一位为小数点后一位

### S3 获取当前时间

In [56]:
data.iloc[0]

0          66666
1              0
2             31
3              1
4              1
5              1
6              6
7         Damrey
8    2.01107e+07
Name: 0, dtype: object

In [57]:
str(int(data.iloc[0][8]))

'20110729'

### S4 写入mongodb

In [58]:
#测试
list_startend[0]

[0, 32]

In [52]:
data.iloc[0]

0          66666
1              0
2             31
3              1
4              1
5              1
6              6
7         Damrey
8    2.01107e+07
Name: 0, dtype: object

In [50]:
header,body=getHeaderBody(0+1,32)

In [51]:
header

0          66666
1              0
2             31
3              1
4              1
5              1
6              6
7         Damrey
8    2.01107e+07
Name: 0, dtype: object

### S5 遍历起始list，自动获取header以及body

In [59]:
typhoon_list=[]
for start in list_startend:
    start_index=start[0]
    end_index=None
    if len(start)==1:        
        end_index=None
    else:
#         start_index=start[0]
        end_index=start[1]
    # 获取header与body
#     header,body=None
    header,body=getHeaderBody(start_index+1,end_index)
    # 从header中获取name
#     typhoon_code=header[7]
#     typhoon_num=header[3]
#     typhoon_startdate=datetime.strptime(str(int(header[8]),'%Y%m%d'))
    
#     print(header)
#     print(f"code:{typhoon_code}|num:{typhoon_num}|typhoon_date:{str(int(header[8]))}")
    
#     print(f"当前body长度{len(body)}")
    for i in range(len(body)):
        if i>=len(body)-1:
            print("跳出本次循环")
            break
        temp_typhoon=body.iloc[i] 
        print(f"当前i:{i}")
        print(f"当前index:{start_index+1+i}")
        print(f"当前header:\n{header}")
        print(f"当前body:\n{body.iloc[i+1]}")
        print(f"写入的台风code为{header[7]}!!")
        typhoon_list.append(convert2Typhoon(body.iloc[i+1],header[7],header[3]))
    print('--------')

当前i:0
当前index:1
当前header:
0          66666
1              0
2             31
3              1
4              1
5              1
6              6
7         Damrey
8    2.01107e+07
Name: 0, dtype: object
当前body:
0    2000050506
1             1
2            99
3          1350
4          1004
5            15
Name: 2, dtype: int64
写入的台风code为Damrey!!
当前i:1
当前index:2
当前header:
0          66666
1              0
2             31
3              1
4              1
5              1
6              6
7         Damrey
8    2.01107e+07
Name: 0, dtype: object
当前body:
0    2000050512
1             1
2           102
3          1344
4          1004
5            15
Name: 3, dtype: int64
写入的台风code为Damrey!!
当前i:2
当前index:3
当前header:
0          66666
1              0
2             31
3              1
4              1
5              1
6              6
7         Damrey
8    2.01107e+07
Name: 0, dtype: object
当前body:
0    2000050518
1             1
2           107
3          1337
4          1002
5            15


当前index:105
当前header:
0          66666
1              0
2             27
3              6
4              3
5              1
6              6
7         Kirogi
8    2.01107e+07
Name: 79, dtype: object
当前body:
0    2000070900
1             3
2           421
3          1465
4           985
5            25
Name: 106, dtype: int64
写入的台风code为Kirogi!!
跳出本次循环
--------
当前i:0
当前index:108
当前header:
0          66666
1              0
2             37
3              7
4              4
5              0
6              6
7        Kai-tak
8    2.01107e+07
Name: 107, dtype: object
当前body:
0    2000070218
1             1
2           157
3          1175
4          1002
5            12
Name: 109, dtype: int64
写入的台风code为Kai-tak!!
当前i:1
当前index:109
当前header:
0          66666
1              0
2             37
3              7
4              4
5              0
6              6
7        Kai-tak
8    2.01107e+07
Name: 107, dtype: object
当前body:
0    2000070300
1             1
2           159
3          1178
4     

当前index:322
当前header:
0          66666
1              0
2             40
3             13
4              9
5              3
6              6
7        Ewiniar
8    2.01107e+07
Name: 289, dtype: object
当前body:
0    2000081706
1             3
2           387
3          1500
4           985
5            25
Name: 323, dtype: int64
写入的台风code为Ewiniar!!
当前i:33
当前index:323
当前header:
0          66666
1              0
2             40
3             13
4              9
5              3
6              6
7        Ewiniar
8    2.01107e+07
Name: 289, dtype: object
当前body:
0    2000081712
1             2
2           385
3          1497
4           990
5            23
Name: 324, dtype: int64
写入的台风code为Ewiniar!!
当前i:34
当前index:324
当前header:
0          66666
1              0
2             40
3             13
4              9
5              3
6              6
7        Ewiniar
8    2.01107e+07
Name: 289, dtype: object
当前body:
0    2000081718
1             2
2           384
3          1500
4           992
5 

写入的台风code为Sonamu!!
当前i:4
当前index:526
当前header:
0          66666
1              0
2             16
3             21
4             17
5              1
6              6
7         Sonamu
8    2.01107e+07
Name: 521, dtype: object
当前body:
0    2000091600
1             3
2           249
3          1418
4           980
5            30
Name: 527, dtype: int64
写入的台风code为Sonamu!!
当前i:5
当前index:527
当前header:
0          66666
1              0
2             16
3             21
4             17
5              1
6              6
7         Sonamu
8    2.01107e+07
Name: 521, dtype: object
当前body:
0    2000091606
1             4
2           261
3          1417
4           975
5            33
Name: 528, dtype: int64
写入的台风code为Sonamu!!
当前i:6
当前index:528
当前header:
0          66666
1              0
2             16
3             21
4             17
5              1
6              6
7         Sonamu
8    2.01107e+07
Name: 521, dtype: object
当前body:
0    2000091612
1             4
2           273
3          14

<GeoTyphoonRealData: code:Damrey|num:0001|date:2000-05-05 06:00:00|bp:1004.0|wsm:15.0|level:1|latlon:[135.0, 9.9]>

In [26]:
# 查看一下新加了num的
type(typhoon_list[0].num)

str

In [27]:
# 写入mongodb
index_save=0
for temp in typhoon_list:
    try:
        temp.save()
        index_save=index_save+1
        print(f'{index_save}写入成功')
    except ValidationError as ex:
        print(f'出现错误的位置{index_save}|err:{str(ex)}')

1写入成功
2写入成功
3写入成功
4写入成功
5写入成功
6写入成功
7写入成功
8写入成功
9写入成功
10写入成功
11写入成功
12写入成功
13写入成功
14写入成功
15写入成功
16写入成功
17写入成功
18写入成功
19写入成功
20写入成功
21写入成功
22写入成功
23写入成功
24写入成功
25写入成功
26写入成功
27写入成功
28写入成功
29写入成功
30写入成功
31写入成功
32写入成功
33写入成功
34写入成功
35写入成功
36写入成功
37写入成功
38写入成功
39写入成功
40写入成功
41写入成功
42写入成功
43写入成功
44写入成功
45写入成功
46写入成功
47写入成功
48写入成功
49写入成功
50写入成功
51写入成功
52写入成功
53写入成功
54写入成功
55写入成功
56写入成功
57写入成功
58写入成功
59写入成功
60写入成功
61写入成功
62写入成功
63写入成功
64写入成功
65写入成功
66写入成功
67写入成功
68写入成功
69写入成功
70写入成功
71写入成功
72写入成功
73写入成功
74写入成功
75写入成功
76写入成功
77写入成功
78写入成功
79写入成功
80写入成功
81写入成功
82写入成功
83写入成功
84写入成功
85写入成功
86写入成功
87写入成功
88写入成功
89写入成功
90写入成功
91写入成功
92写入成功
93写入成功
94写入成功
95写入成功
96写入成功
97写入成功
98写入成功
99写入成功
100写入成功
101写入成功
102写入成功
103写入成功
104写入成功
105写入成功
106写入成功
107写入成功
108写入成功
109写入成功
110写入成功
111写入成功
112写入成功
113写入成功
114写入成功
115写入成功
116写入成功
117写入成功
118写入成功
119写入成功
120写入成功
121写入成功
122写入成功
123写入成功
124写入成功
125写入成功
126写入成功
127写入成功
128写入成功
129写入成功
130写入成功
131写入成功
132写入成功
133写入成功
134写入成功
135写入成功
136写入成功
137写入成功
138写入成功
139写

722写入成功
723写入成功
724写入成功
725写入成功
726写入成功
727写入成功
728写入成功
729写入成功
730写入成功
731写入成功
732写入成功
733写入成功
734写入成功
735写入成功
736写入成功
737写入成功
738写入成功
739写入成功
740写入成功
741写入成功
742写入成功
743写入成功
744写入成功
745写入成功
746写入成功
747写入成功
748写入成功
749写入成功
750写入成功
751写入成功
752写入成功
753写入成功
754写入成功
755写入成功


In [28]:
typhoon_list[1].code

'(nameless)'

In [29]:
typhoon_list[197].code

'(nameless)(-)1'

In [30]:
typhoon_list[678].code

'Jean'

In [1]:
# 最后修改 2019-04-09

In [1]:
from mongoengine import *
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime

使用** mongoengine **  
[connect](http://docs.mongoengine.org/guide/connecting.html#guide-connecting)进行数据库连接

In [2]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## S1 获取指定目录下的全部文件名称

In [9]:
targetfilename=r"CH1956BST.txt"
# mac 15
# targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data"
targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/data/typhoon"
# aw15
# targetpath=r"D:\01proj\typhoon\TyphoonSearchSys\demo_data"
# p52s
# targetpath=r"D:\04git仓库\TyphoonSearchSys\demo_data"
# w540
# targetpath=r"D:\02git仓库\TyphoonSearchSys\demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [13]:
list_files=os.listdir(targetpath)
list_files[:5]

['CH1998BST.txt',
 'CH1986BST.txt',
 'CH1994BST.txt',
 'CH1951BST.txt',
 'CH2012BST.txt']

获取以.txt结尾的

## 1 读取文件

### 读取方式1：使用pandas

In [5]:
fullname

'D:\\02git仓库\\TyphoonSearchSys\\demo_data\\CH1956BST.txt'

不使用pandas的read_table

In [6]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\s+',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


--------

## 2 判断标志位的方式（必须同时满足），并获取标志位所在位置的数组（mark_indexs）

1-第一位是否为66666

2- 第7位是否不为Nan

In [7]:
data.iloc[0][0]==66666 and data.iloc[0][7]!=np.nan

True

### 写成方法：

In [8]:
def checkMark(index):
    '''
        判断是否为标志位
    '''
    return data.iloc[index][0]==66666 and data.iloc[index][7]!=np.nan

找到标志位所在的位置

In [9]:
mark_indexs=[]
for i in range(len(data)):
#     print(i)
    if checkMark(i):
        mark_indexs.append(i)

--------

## 3 获取观测值并写入mongoDB（暂时先写在list中）

In [10]:
# 保存起止数组的数组
list_startend=[]
index=0
for val in mark_indexs:
#     list_startend.append(mark_indexs[index:2])
#     print(mark_indexs[index:index+2])
    list_startend.append(mark_indexs[index:index+2])
    index=index+1
#     print(index)

[[789, 830], [830]]  
注意最后的数组是长度为1的数组，因为最后一次出现标志位后出现的都是观测数据，便结束了

----

## 4 使用mongoEngine

In [11]:
from mongoengine import *

In [12]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

定义 mongodb中的存储对象（ORM）

In [13]:
class GeoTyphoonRealData(Document):
    '''
        支持geojson的存储至mongodb的model
    '''
    code=StringField(max_length=10)
    num=StringField()
    date=DateTimeField()
    bp=FloatField()
    wsm=FloatField()
    # 注意此处与django中的类型不同，django的类型为IntegerField，mongoengine为IntField！
    level=IntField()
    # latlon=models.ForeignKey(Point,on_delete=models.CASCADE)
    latlon=PointField()
    meta = {'collection': 'geotyphoonrealdata'}
    
    def __str__(self):
        return f'code:{self.code}|num:{self.num}|date:{self.date}|bp:{self.bp}|wsm:{self.wsm}|level:{self.level}|latlon:{self.latlon}'

In [14]:
import datetime

** 提示错误如下 **:  
NotUniqueError: Tried to save duplicate unique keys (E11000 duplicate key error collection: typhoon.Typhoon_geotyphoonrealdata index: __primary_key__ dup key: { : null })  
是mongodb中的对应document中可能设置了__primary_key__，我的解决办法是 ** 删除掉 ** 

### S1 根据标志位遍历df

循环遍历list_startend

In [15]:
for val in list_startend:
    if len(val)>1:        
        print(f"起始{val[0]}——终止{val[1]-1}")
    elif len(val)==1:
        print(f"起始{val[0]}")

起始0——终止10
起始11——终止58
起始59——终止80
起始81——终止117
起始118——终止126
起始127——终止148
起始149——终止158
起始159——终止168
起始169——终止186
起始187——终止196
起始197——终止218
起始219——终止223
起始224——终止270
起始271——终止286
起始287——终止298
起始299——终止309
起始310——终止323
起始324——终止335
起始336——终止371
起始372——终止384
起始385——终止396
起始397——终止428
起始429——终止463
起始464——终止504
起始505——终止520
起始521——终止535
起始536——终止586
起始587——终止622
起始623——终止659
起始660——终止680
起始681——终止697
起始698——终止744
起始745——终止803
起始804——终止817
起始818——终止831
起始832——终止850
起始851——终止865
起始866


In [16]:
def getHeaderBody(start,end=None):
    '''
        根据df的起止位置截取其中的头部（66666	0	25	0	1	0	6.0	(nameless)	20180501.0），
        以及数据内容body（2017041406	0	109	1302	1010	10）
    '''
    header=data.iloc[start-1]
    body=data.iloc[start:end,0:6]
    return header,body

In [17]:
from datetime import datetime

- 19-04-03 此处加入一个新的需求，加入对于 **台风 数字编号 **的获取

In [18]:
# data.iloc[0]

In [21]:
def convert2Typhoon(obj,code,num):
    '''
        根据传入的series将其转为typhoob Model
    '''
    lat=float(f"{str(obj[2])[:-1]}.{str(obj[2])[-1:]}")
    lon=float(f"{str(obj[3])[:-1]}.{str(obj[3])[-1:]}")
    stamp_str=obj[0]
    # TODO 注意此处需要修改num的值
    stamp=datetime.strptime(str(stamp_str),'%Y%m%d%H')
    num_str=num
    typhoon_temp=GeoTyphoonRealData(code=code,
                                date=stamp,
                                num=num_str,
                                bp=obj[4],
                                wsm=obj[5],
                                level=obj[1],
                                latlon=[lon,lat])
    return typhoon_temp

### S2 拼接经纬度
获取经纬度：  
    - 将字符串转成float，并且最后一位为小数点后一位

### S3 获取当前时间

In [20]:
data.iloc[0]

0          66666
1              0
2             10
3              1
4              0
5              0
6              6
7     (nameless)
8    2.01107e+07
Name: 0, dtype: object

In [21]:
str(int(data.iloc[0][8]))

'20110729'

### S4 写入mongodb

### S5 遍历起始list，自动获取header以及body

In [22]:
typhoon_list=[]
for start in list_startend:
    start_index=start[0]
    end_index=None
    if len(start)==1:        
        end_index=None
    else:
#         start_index=start[0]
        end_index=start[1]
    # 获取header与body
#     header,body=None
    header,body=getHeaderBody(start_index+1,end_index)
    # 从header中获取name
#     typhoon_code=header[7]
#     typhoon_num=header[3]
#     typhoon_startdate=datetime.strptime(str(int(header[8]),'%Y%m%d'))
    
#     print(header)
#     print(f"code:{typhoon_code}|num:{typhoon_num}|typhoon_date:{str(int(header[8]))}")
    
#     print(f"当前body长度{len(body)}")
    for i in range(len(body)):
        if i>=len(body)-1:
            print("跳出本次循环")
            break
        temp_typhoon=body.iloc[i] 
        print(f"当前i:{i}")
        print(f"当前index:{start_index+1+i}")
        print(f"当前header:\n{header}")
        print(f"当前body:\n{body.iloc[i+1]}")
        print(f"写入的台风code为{header[7]}!!")
        typhoon_list.append(convert2Typhoon(body.iloc[i+1],header[7],header[4]))
    print('--------')

当前i:0
当前index:1
当前header:
0          66666
1              0
2             10
3              1
4           1401
5              0
6              6
7       Lingling
8    2.01503e+07
Name: 0, dtype: object
当前body:
0    2014011712
1             1
2            97
3          1276
4          1006
5            13
Name: 2, dtype: int64
写入的台风code为Lingling!!
当前i:1
当前index:2
当前header:
0          66666
1              0
2             10
3              1
4           1401
5              0
6              6
7       Lingling
8    2.01503e+07
Name: 0, dtype: object
当前body:
0    2014011718
1             1
2            96
3          1274
4          1006
5            13
Name: 3, dtype: int64
写入的台风code为Lingling!!
当前i:2
当前index:3
当前header:
0          66666
1              0
2             10
3              1
4           1401
5              0
6              6
7       Lingling
8    2.01503e+07
Name: 0, dtype: object
当前body:
0    2014011800
1             1
2            94
3          1275
4          1002
5           

ValueError: day is out of range for month

In [23]:
typhoon_list

[<GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-18 06:00:00|bp:1004.0|wsm:12.0|level:1|latlon:[116.0, 13.5]>,
 <GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-18 12:00:00|bp:1002.0|wsm:12.0|level:1|latlon:[116.6, 14.4]>,
 <GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-18 18:00:00|bp:1002.0|wsm:15.0|level:1|latlon:[116.9, 15.2]>,
 <GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-19 00:00:00|bp:1002.0|wsm:15.0|level:1|latlon:[116.9, 16.2]>,
 <GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-19 06:00:00|bp:1002.0|wsm:15.0|level:1|latlon:[117.3, 17.3]>,
 <GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-19 12:00:00|bp:1004.0|wsm:12.0|level:1|latlon:[117.8, 17.7]>,
 <GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-19 18:00:00|bp:1006.0|wsm:12.0|level:1|latlon:[118.6, 18.0]>,
 <GeoTyphoonRealData: code:(nameless)|num:5601|date:1956-01-20 00:00:00|bp:1006.0|wsm:12.0|level:1|latlon:[119.5, 18.3]>,
 <GeoTyphoonRealData: co

In [26]:
# 查看一下新加了num的
type(typhoon_list[0].num)

str

In [27]:
# 写入mongodb
index_save=0
for temp in typhoon_list:
    try:
        temp.save()
        index_save=index_save+1
        print(f'{index_save}写入成功')
    except ValidationError as ex:
        print(f'出现错误的位置{index_save}|err:{str(ex)}')

1写入成功
2写入成功
3写入成功
4写入成功
5写入成功
6写入成功
7写入成功
8写入成功
9写入成功
10写入成功
11写入成功
12写入成功
13写入成功
14写入成功
15写入成功
16写入成功
17写入成功
18写入成功
19写入成功
20写入成功
21写入成功
22写入成功
23写入成功
24写入成功
25写入成功
26写入成功
27写入成功
28写入成功
29写入成功
30写入成功
31写入成功
32写入成功
33写入成功
34写入成功
35写入成功
36写入成功
37写入成功
38写入成功
39写入成功
40写入成功
41写入成功
42写入成功
43写入成功
44写入成功
45写入成功
46写入成功
47写入成功
48写入成功
49写入成功
50写入成功
51写入成功
52写入成功
53写入成功
54写入成功
55写入成功
56写入成功
57写入成功
58写入成功
59写入成功
60写入成功
61写入成功
62写入成功
63写入成功
64写入成功
65写入成功
66写入成功
67写入成功
68写入成功
69写入成功
70写入成功
71写入成功
72写入成功
73写入成功
74写入成功
75写入成功
76写入成功
77写入成功
78写入成功
79写入成功
80写入成功
81写入成功
82写入成功
83写入成功
84写入成功
85写入成功
86写入成功
87写入成功
88写入成功
89写入成功
90写入成功
91写入成功
92写入成功
93写入成功
94写入成功
95写入成功
96写入成功
97写入成功
98写入成功
99写入成功
100写入成功
101写入成功
102写入成功
103写入成功
104写入成功
105写入成功
106写入成功
107写入成功
108写入成功
109写入成功
110写入成功
111写入成功
112写入成功
113写入成功
114写入成功
115写入成功
116写入成功
117写入成功
118写入成功
119写入成功
120写入成功
121写入成功
122写入成功
123写入成功
124写入成功
125写入成功
126写入成功
127写入成功
128写入成功
129写入成功
130写入成功
131写入成功
132写入成功
133写入成功
134写入成功
135写入成功
136写入成功
137写入成功
138写入成功
139写

722写入成功
723写入成功
724写入成功
725写入成功
726写入成功
727写入成功
728写入成功
729写入成功
730写入成功
731写入成功
732写入成功
733写入成功
734写入成功
735写入成功
736写入成功
737写入成功
738写入成功
739写入成功
740写入成功
741写入成功
742写入成功
743写入成功
744写入成功
745写入成功
746写入成功
747写入成功
748写入成功
749写入成功
750写入成功
751写入成功
752写入成功
753写入成功
754写入成功
755写入成功


In [28]:
typhoon_list[1].code

'(nameless)'

In [29]:
typhoon_list[197].code

'(nameless)(-)1'

In [30]:
typhoon_list[678].code

'Jean'